In [1]:
from math import *
from leg import *
from body import *
from control import *
import numpy as np

In [2]:
b = Body()
c = Control(b)
c.build_quad()
c.set_named_posture('stand')
c.walk(8, 45)
print('Done!')

PermissionError: [Errno 13] Permission denied: '/dev/i2c-1'